In [1]:
import pandas as pd
import numpy as np



In [2]:
TRAIN_PATH = "traindata_imputed.csv"   # update if needed
TEST_PATH  = "testdata_preprocessed.csv"


# -----------------------------
# LOAD
# -----------------------------
df_tr = pd.read_csv(TRAIN_PATH)
df_te = pd.read_csv(TEST_PATH)

In [3]:
cols_to_drop = [
    "valeur_CO_was_missing",
    "valeur_NO2_was_missing",
    "valeur_O3_was_missing",
    "valeur_PM10_was_missing",
    "valeur_PM25_was_missing"
]

df_tr = df_tr.drop(columns=cols_to_drop, errors="ignore")


In [5]:
df_tr.head(4)


,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_weekend,temperature_2m,relative_humidity_2m,...,et0_fao_evapotranspiration,snowfall,rain,showers,weather_code,lockdown_code,datetime,hour,dayofweek,month
0,2020-01-01 00,42.9,0.7180,15.7,73.1,64.4,1,0,0.9,100,...,0.0,0.0,0.0,0.0,3,0,2020-01-01 00:00:00,0,2,1
1,2020-01-01 01,33.6,0.5870,10.1,74.8,66.0,1,0,-0.1,99,...,0.0,0.0,0.0,0.0,3,0,2020-01-01 01:00:00,1,2,1
2,2020-01-01 02,29.3,0.4165,5.1,51.0,44.9,1,0,2.6,98,...,0.0,0.0,0.0,0.0,3,0,2020-01-01 02:00:00,2,2,1
3,2020-01-01 03,30.5,0.2460,7.2,27.7,25.1,1,0,2.1,100,...,0.0,0.0,0.0,0.0,3,0,2020-01-01 03:00:00,3,2,1


In [6]:


# =========================
# CONFIG
# =========================
TIME = "datetime"
RAW_ID = "id"  # might be wrong in your data; we will auto-fix below

TARGETS = ["valeur_NO2","valeur_CO","valeur_O3","valeur_PM10","valeur_PM25"]

WEATHER = [
    'temperature_2m','relative_humidity_2m','dew_point_2m','apparent_temperature',
    'pressure_msl','wind_speed_10m','wind_direction_10m','precipitation',
    'cloud_cover','cloud_cover_low','cloud_cover_mid','cloud_cover_high',
    'shortwave_radiation','direct_radiation','diffuse_radiation','global_tilted_irradiance',
    'wind_speed_80m','wind_speed_120m','wind_speed_180m',
    'wind_direction_80m','wind_direction_120m','wind_direction_180m',
    'wind_gusts_10m','vapour_pressure_deficit','cape','evapotranspiration',
    'et0_fao_evapotranspiration','snowfall','rain','showers','weather_code',
    'visibility','lockdown_code'
]
STATIC_KNOWN = ["is_holiday","is_weekend","lockdown_code"]

LAGS  = [6, 12]
ROLLS = [6, 24]   # hours

# =========================
# MAIN
# =========================
def build_dataset(df_tr: pd.DataFrame, df_te: pd.DataFrame):
    # --- Tidy/union ---
    df_tr = df_tr.copy()
    df_te = df_te.copy()
    df_tr[TIME] = pd.to_datetime(df_tr[TIME])
    df_te[TIME] = pd.to_datetime(df_te[TIME])
    df_tr["__split__"] = "train"; df_te["__split__"] = "test"
    df = pd.concat([df_tr, df_te], ignore_index=True)
    df = df.sort_values([TIME]).reset_index(drop=True)

    # --- Choose grouping key ---
    # If RAW_ID repeats, use it. Otherwise create a single group "__grp__" so lags/rolls work.
    use_id = RAW_ID if RAW_ID in df.columns else None
    if use_id is not None:
        nunq = df[RAW_ID].nunique()
        if nunq == len(df):  # all unique -> useless for groupby
            use_id = None
    if use_id is None:
        df["__grp__"] = "all"
        GROUP = "__grp__"
    else:
        GROUP = RAW_ID
        # make sure it's clean type
        df[GROUP] = df[GROUP].astype(str).str.strip()

    # --- Which weather cols exist ---
    weather_cols = [c for c in WEATHER if c in df.columns]

    # --- Mark future test horizon (first row per group where ANY target is NaN) ---
    te = df[df["__split__"] == "test"]
    if len(te):
        # compute per-group first future timestamp
        tmp = te.copy()
        tmp["_nan_any"] = tmp[TARGETS].isna().any(axis=1)
        first_future = (
            tmp[tmp["_nan_any"]]
            .groupby(GROUP, as_index=True)[TIME]
            .min()
        )
        df = df.join(first_future.rename("__future_start__"), on=GROUP)
        df["__is_future_test__"] = (df["__split__"]=="test") & df["__future_start__"].notna() & (df[TIME] >= df["__future_start__"])
        df.drop(columns="__future_start__", inplace=True)
    else:
        df["__is_future_test__"] = False

    # --- Time features ---
    df["is_day"] = ((df["hour"] >= 6) & (df["hour"] < 18)).astype(int)
    df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)
    df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)
    df["dow"] = df[TIME].dt.dayofweek
    df["dow_sin"] = np.sin(2*np.pi*df["dow"]/7)
    df["dow_cos"] = np.cos(2*np.pi*df["dow"]/7)

    # --- Mask future test weather (rule) ---
    if weather_cols:
        df.loc[df["__is_future_test__"], weather_cols] = np.nan

    # --- Build features into a dict (avoid fragmentation) ---
    feats = {}

    # Helper: add series to dict safely
    def add_feat(name, series):
        feats[name] = series

    # POLLUTANT lags/rollings (past only)
    for col in TARGETS:
        g = df.groupby(GROUP)[col]
        for l in LAGS:
            add_feat(f"{col}_lag_{l}", g.shift(l))
        s = g.shift(1)  # past-only base
        for w in ROLLS:
            add_feat(f"{col}_roll_mean_{w}", s.groupby(df[GROUP]).rolling(w, min_periods=1).mean().reset_index(level=0, drop=True))
            add_feat(f"{col}_roll_std_{w}",  s.groupby(df[GROUP]).rolling(w, min_periods=2).std().reset_index(level=0, drop=True))

    # Cross-lags (examples)
    add_feat("NO2_lag1_for_O3", df.groupby(GROUP)["valeur_NO2"].shift(1))
    add_feat("PM10_lag1_for_PM25", df.groupby(GROUP)["valeur_PM10"].shift(1))

    # WEATHER lags/rollings (past only, after masking)
    for col in weather_cols:
        g = df.groupby(GROUP)[col]
        for l in LAGS:
            add_feat(f"{col}_lag_{l}", g.shift(l))
        s = g.shift(1)
        for w in ROLLS:
            add_feat(f"{col}_roll_mean_{w}", s.groupby(df[GROUP]).rolling(w, min_periods=1).mean().reset_index(level=0, drop=True))
            # std only if numeric
            if pd.api.types.is_numeric_dtype(df[col]):
                add_feat(f"{col}_roll_std_{w}",  s.groupby(df[GROUP]).rolling(w, min_periods=2).std().reset_index(level=0, drop=True))

    # Concatenate all features at once (fast, no fragmentation)
    feat_df = pd.DataFrame(feats, index=df.index)

    # --- Assemble full X with calendar/static features ---
    base_cols = ["hour","is_day","hour_sin","hour_cos","dow","dow_sin","dow_cos"] + [c for c in STATIC_KNOWN if c in df.columns]
    X_all = pd.concat([feat_df, df[base_cols]], axis=1)

    # --- Impute features (ffill per group → train medians; std→0) ---
    X_all = X_all.groupby(df[GROUP]).ffill()

    train_meds = X_all[df["__split__"]=="train"].median(numeric_only=True)
    X_all = X_all.fillna(train_meds)

    std_cols = [c for c in X_all.columns if "_roll_std_" in c]
    if std_cols:
        X_all[std_cols] = X_all[std_cols].fillna(0.0)

    # --- Outputs ---
    feat_cols = list(X_all.columns)
    meta = df[[GROUP, TIME, "__split__", "__is_future_test__"]].copy()
    Y_all = df[TARGETS].copy()

    # Train
    X_train = X_all[meta["__split__"]=="train"].reset_index(drop=True)
    Y_train = Y_all[meta["__split__"]=="train"].reset_index(drop=True)

    # Test rows to predict (future only)
    mask_pred = (meta["__split__"]=="test") & (meta["__is_future_test__"])
    X_test_pred = X_all[mask_pred].reset_index(drop=True)
    meta_test = df.loc[mask_pred, [GROUP, TIME]].rename(columns={GROUP:"id"}).reset_index(drop=True)

    print(f"Groups used: {GROUP} ({df[GROUP].nunique()} unique)")
    print("X_train:", X_train.shape, "| Y_train:", Y_train.shape)
    print("X_test_pred:", X_test_pred.shape, "| meta_test:", meta_test.shape)
    print("Num features:", len(feat_cols))

    return X_train, Y_train, X_test_pred, meta_test, feat_cols






In [7]:
X_train, Y_train, X_test_pred, meta_test, feat_cols = build_dataset(df_tr, df_te)


Groups used: __grp__ (1 unique)
X_train: (40991, 186) | Y_train: (40991, 5)
X_test_pred: (504, 186) | meta_test: (504, 2)
Num features: 186


In [8]:

X_train

,valeur_NO2_lag_6,valeur_NO2_lag_12,valeur_NO2_roll_mean_6,valeur_NO2_roll_std_6,valeur_NO2_roll_mean_24,valeur_NO2_roll_std_24,valeur_CO_lag_6,valeur_CO_lag_12,valeur_CO_roll_mean_6,valeur_CO_roll_std_6,...,hour,is_day,hour_sin,hour_cos,dow,dow_sin,dow_cos,is_holiday,is_weekend,lockdown_code
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.000000,1.000000e+00,2,0.974928,-0.222521,1,0,0.0
1,NaN,NaN,42.900000,NaN,42.900000,NaN,NaN,NaN,0.718000,NaN,...,1,0,0.258819,9.659258e-01,2,0.974928,-0.222521,1,0,0.0
2,NaN,NaN,38.250000,6.576093,38.250000,6.576093,NaN,NaN,0.652500,0.092631,...,2,0,0.500000,8.660254e-01,2,0.974928,-0.222521,1,0,0.0
3,NaN,NaN,35.266667,6.951499,35.266667,6.951499,NaN,NaN,0.573833,0.151181,...,3,0,0.707107,7.071068e-01,2,0.974928,-0.222521,1,0,0.0
4,NaN,NaN,34.075000,6.155959,34.075000,6.155959,NaN,NaN,0.491875,0.205197,...,4,0,0.866025,5.000000e-01,2,0.974928,-0.222521,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,15.353253,34.834996,14.639492,1.378732,17.575845,6.620898,0.173,0.215,0.179833,0.013045,...,18,0,-1.000000,-1.836970e-16,1,0.781831,0.623490,0,0,0.0
40987,13.436155,31.917330,15.235393,2.247730,17.779527,6.582483,0.170,0.198,0.188000,0.020890,...,19,0,-0.965926,2.588190e-01,1,0.781831,0.623490,0,0,0.0
40988,13.291387,26.459556,17.576349,5.274854,18.238884,6.864820,0.172,0.187,0.200500,0.028877,...,20,0,-0.866025,5.000000e-01,1,0.781831,0.623490,0,0,0.0
40989,14.056240,25.102253,20.023399,6.211857,18.706749,7.135865,0.177,0.183,0.210833,0.027709,...,21,0,-0.707107,7.071068e-01,1,0.781831,0.623490,0,0,0.0


In [9]:
X_test_pred

,valeur_NO2_lag_6,valeur_NO2_lag_12,valeur_NO2_roll_mean_6,valeur_NO2_roll_std_6,valeur_NO2_roll_mean_24,valeur_NO2_roll_std_24,valeur_CO_lag_6,valeur_CO_lag_12,valeur_CO_roll_mean_6,valeur_CO_roll_std_6,...,hour,is_day,hour_sin,hour_cos,dow,dow_sin,dow_cos,is_holiday,is_weekend,lockdown_code
0,16.958790,15.540546,24.047803,6.241142,19.667895,7.566469,0.205,0.179,0.219167,0.021757,...,23,0,-0.258819,9.659258e-01,1,0.781831,0.623490,0,0,0.0
1,18.928662,15.353253,25.465606,5.797723,19.975450,7.581587,0.222,0.173,0.222000,0.023054,...,0,0,0.000000,1.000000e+00,2,0.974928,-0.222521,0,0,0.0
2,27.481890,13.436155,27.099841,5.197429,20.463940,7.380240,0.245,0.170,0.222000,0.026621,...,1,0,0.258819,9.659258e-01,2,0.974928,-0.222521,0,0,0.0
3,27.973688,13.291387,26.972492,6.357877,20.957504,7.180812,0.234,0.172,0.214333,0.026652,...,2,0,0.500000,8.660254e-01,2,0.974928,-0.222521,0,0,0.0
4,32.770370,14.056240,26.471894,8.907391,21.410790,7.052377,0.225,0.177,0.204500,0.028991,...,3,0,0.707107,7.071068e-01,2,0.974928,-0.222521,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,20.173417,20.173417,20.173417,8.907391,20.173417,8.907391,0.184,0.184,0.184000,0.028991,...,18,0,-1.000000,-1.836970e-16,1,0.781831,0.623490,0,0,0.0
500,20.173417,20.173417,20.173417,8.907391,20.173417,8.907391,0.184,0.184,0.184000,0.028991,...,19,0,-0.965926,2.588190e-01,1,0.781831,0.623490,0,0,0.0
501,20.173417,20.173417,20.173417,8.907391,20.173417,8.907391,0.184,0.184,0.184000,0.028991,...,20,0,-0.866025,5.000000e-01,1,0.781831,0.623490,0,0,0.0
502,20.173417,20.173417,20.173417,8.907391,20.173417,8.907391,0.184,0.184,0.184000,0.028991,...,21,0,-0.707107,7.071068e-01,1,0.781831,0.623490,0,0,0.0
